<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_3_EXTRACTING_LOCATIONS_AND_DATES_FROM_RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
!pip install sddk
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:
PHI_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zfTw0Hf304maBmrYvaMxRLnv1zfAVFixrtGTTsLCcT4/edit?usp=sharing")

# Read data

In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_enriched_raw.json", "df", conf)

In [40]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,[/text/237443?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 169,3]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 169,3]",[Att. — Athens: Akropolis — stoich. 28 — 303/2...,{},[4],[1\n\n\n],[ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πί...,[CSCA.csv],[237443]
1,[/text/237444?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 169,4]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 169,4]",[Att. — Athens: EM — stoich. 35 — 306-302 BC],{},[15],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15],[[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρ...,[CSCA.csv],[237444]
2,[/text/237445?location=1365&patt=&bookid=411&b...,[CSCA],"[5 (1972) 173,5]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n5 (1972) 173,5]",[Att. — Athens — non-stoich.],{},[16],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n],[— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9...,[CSCA.csv],[237445]
3,[/text/237446?location=1365&patt=&bookid=411&b...,[CSCA],"[9 (1976) 9,4]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n9 (1976) 9,4]",[Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26....,{},[2],[A.1\nB.1],[ψῆφος ⋮ δημοσία.\nΔ ․],[CSCA.csv],[237446]
4,[/text/237447?location=1365&patt=&bookid=411&b...,[CSCA],"[9 (1976) 11,16]",[Regions\nAttica (IG I-III)\nAttica],"[CSCA\n9 (1976) 11,16]",[Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26....,{},[2],[A.1\nB.1],[ψῆφος ⋮ δημοσία.\nΓ Ε],[CSCA.csv],[237447]


In [0]:
def lists_to_values(list_or_value):
  if isinstance(list_or_value, list):
    value = list_or_value[0]
  else: 
    value = list_or_value
  return value
for column in PHI.columns:
  PHI[column] = PHI.apply(lambda row: lists_to_values(row[column]), axis=1)

In [14]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/237443?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,3",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,3",Att. — Athens: Akropolis — stoich. 28 — 303/2 BC,{},4,1\n\n\n,ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πίδ...,CSCA.csv,237443
1,/text/237444?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,4",Att. — Athens: EM — stoich. 35 — 306-302 BC,{},15,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15,[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρυ...,CSCA.csv,237444
2,/text/237445?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 173,5",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 173,5",Att. — Athens — non-stoich.,{},16,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n,— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9․...,CSCA.csv,237445
3,/text/237446?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 9,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 9,4",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,{},2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΔ ․,CSCA.csv,237446
4,/text/237447?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 11,16",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 11,16",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,{},2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΓ Ε,CSCA.csv,237447


# Testing extracting info from tildeinfo

In [0]:
# generate sample for testing purposes:
PHI_by_500 = PHI[PHI["PHI_ID"].isin(range(1, 300000, 500))]

In [36]:
PHI_by_500.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
625,/text/262001?location=1040&patt=&bookid=517&of...,Panamara,316,Regions\nAsia Minor\nCaria,Panamara\n316,IStr 427,{},2,1\n,"Διῒ Πανημέρῳ, κόμαι Εὐτύχεως· ἐπὶ ἱερέω[ς] Κλα...",Panamara.csv,262001
912,/text/266001?location=1673&patt=&bookid=587&of...,St.Pont. III,110a,Regions\nAsia Minor\nPontus and Paphlagonia,St.Pont. III\n110a,Pont. — Amasia — Rom. Imp. period,{},6,1\n\n\n\n5\n,Αὐρ(ηλίῳ) Φιλο-\nμούσῳ ❦\nἀρχιάτρῳ\nῬω[μᾶνα(?)...,St.Pont.-III.csv,266001
1260,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,{},149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001
1944,/text/79501?location=126&patt=&bookid=23&offse...,"IG XII,8",311,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,8\n311",Thasos,{},7,1\n\n\n\n5\n\n,Ἐπικράτης Κτησιφῶντος\nΠυθίων Περιθύμου\nΠίν...,IG-XII-8.csv,79501
2502,/text/218501?location=1497&patt=&bookid=367&of...,Koptos à Kosseir,163,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Koptos à Kosseir\n163,Eg. — el-Boueib,{},2,1\n,Κάλχων̣\nΠανὶ Χρ(υσόδωτηι).,Koptos---Kosseir.csv,218501


In [0]:
PHI_by_500["info"]

In [43]:
# list of tildeinfos from this sample for development parsing functions
tildeinfos_list = PHI_by_500["tildeinfo"].tolist()
print(tildeinfos_list)

['IStr 427', 'Pont. — Amasia — Rom. Imp. period', 'Att. — Athens: Agora — 220/19', 'Thasos', 'Eg. — el-Boueib', 'Korinthia — Korinthos — ca. 100-150 AD', 'Korinthia — Korinthos — 267-668 AD', 'N. Black Sea — Pantikapaion (Kerch) — 1st c. BC — IosPE IV 253', 'N. Black Sea — Pantikapaion (Kerch) — Roman period? — IosPE IV 348', 'N. Black Sea — Tanais — 155 AD — IosPE II 438', 'Bith. — Prusa ad Olympum (Bursa) — 199 AD', 'Dacia Sup. — Tibiscum (Jupa) — 2nd/3rd c. AD', 'Mys.: Aisepos — Zeus Olbios bei Buğdaylı? — JHS 25,1905,56 Nr.4', 'Makedonia (Bottiaia) — Beroia — ca. 100-150 AD — SEG 35.714', 'Makedonia (Bottiaia) — Beroia — Rachi — 1st c. BC — ABSA 39 (1938/39) 95, 4', 'Delos — ca. 300-250 BC', 'Delos — beg. 2nd c. BC', 'Att. — Athens: Agora — stoich. 29 — 301/0-295/4 a. — *Hesp. 13.1944.242,7 — *SEG 24.119; 29.93', 'Eg. — Thebes [W.]: Syringes', 'Eg. — Thebes [W.]: Syringes', 'Eg. — Thebes [W.]: Syringes', 'Eg. — Thebes [W.]: Syringes', 'Crete, W. — Lappa — 1st c. AD', 'Eg. — Akoris 

In [0]:
def date_extractor(date_value):
    ### simple cleaning?
    datation = re.sub("\/\d+", "", date_value)
    for element in ["?"]:
        datation = datation.replace(element, "")
    for element in ["AD", "Rom. Imp.", "Byzant", "Roman period", "Christian"]:
        if element in datation:
            for element_cent in [("1st", 1, 100),
                            ("2nd", 101, 200),
                            ("3rd", 201, 300),
                            ("4th", 301, 400),
                           ]:
                if element_cent[0] in datation:
                    dating = [element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "AD"]
                    break
                else:
                    try:
                        post = re.search('(after\s|post\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        post = int(post[1])
                        dating = [post, None, None, None, "post", "AD"]
                    except:
                        try:
                            date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(date_both[0])
                            ante = int(date_both[2])
                            dating = [post, ante, None, int(np.average([post, ante])), "range", "AD"]
                        except:
                            try:
                                date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                date_exact = int(date_exact.group(0))
                                dating = [None, None, date_exact, date_exact, "exact", "AD"]
                            except:
                                dating = [None, None, None, None, "unprecise", "AD"]
                                
            break
        else:
            for element_cent in [(" V ", -500, -401), 
                            (" V/IV ", -500, -301),
                            (" IV ", -400, -301),
                            (" III ", -300, -201),
                            (" II ", -200, -101),
                            (" I ", -100, -1),
                            ("5th", -500,  -401),
                            ("4th", -400, -301),
                            ("3rd", -300, -201),
                            ("2nd", -200, -101),
                            ("1st", -100, -1)]:
                if element_cent[0] in datation:
                    ### if there as an element indicating century
                    dating = [element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "BC"]
                    break
                else:
                    try:
                        ante = re.search('(ante\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        ante = int(ante[1]) * -1
                        dating =  [None, ante, None, None, "ante", "BC"]
                    except:
                        try:
                            post = re.search('(post\s|after\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(post[1]) * -1
                            dating = [post, None, None, None, "post", "BC"]
                        except:
                            try:
                                date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                                post = int(date_both[0]) * -1 
                                ante = int(date_both[2]) * -1
                                dating = [post, ante, None, int(np.average([post, ante])), "range", "BC"]
                            except:
                                try:
                                    date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                    date_exact = int(date_exact.group(0)) * -1
                                    dating = [None, None, date_exact, date_exact, "exact", "BC"]
                                except:
                                        dating = [None, None, None, None, "string", "BC"]
    for reign in [("reign of Hadrian", 117, 138),
                  ("reign of Justinian", 527, 565),
                  ("reign of Ant. Pius", 138, 161),
                  ("reign of Augustus", -27, 19),
                  ("reign of Tiberius", 14, 37)]:
        if reign[0] in datation:
            avr = int(np.average([reign[1], reign[2]]))
            if avr < 0:
                period = "BC"
            else:
                period = "AD"
            dating = [reign[1], reign[2], None, avr, "range", period]
            break
    return [date_value] + dating 

In [0]:
def get_date_from_tildeinfo(tildeinfo):
  tildeinfo_list = tildeinfo.split("—")
  datation = tildeinfo_list[-1]
  for el in tildeinfo_list:
    if ("BC" in el) or ("AD" in el):
      datation = el
      break
  dating = date_extractor(datation)
  return dating 

In [65]:
["raw_data", "start", "end", "exact", "avr", "type", "period"]

['raw_data', 'start', 'end', 'exact', 'avr', 'type', 'period']

In [0]:
PHI_by_500["date"] = PHI_by_500.apply(lambda row: get_date_from_tildeinfo(row["tildeinfo"]), axis=1)

In [67]:
PHI_by_500

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,date
625,/text/262001?location=1040&patt=&bookid=517&of...,Panamara,316,Regions\nAsia Minor\nCaria,Panamara\n316,IStr 427,{},2,1\n,"Διῒ Πανημέρῳ, κόμαι Εὐτύχεως· ἐπὶ ἱερέω[ς] Κλα...",Panamara.csv,262001,"[IStr 427, None, None, -427, -427, exact, BC]"
912,/text/266001?location=1673&patt=&bookid=587&of...,St.Pont. III,110a,Regions\nAsia Minor\nPontus and Paphlagonia,St.Pont. III\n110a,Pont. — Amasia — Rom. Imp. period,{},6,1\n\n\n\n5\n,Αὐρ(ηλίῳ) Φιλο-\nμούσῳ ❦\nἀρχιάτρῳ\nῬω[μᾶνα(?)...,St.Pont.-III.csv,266001,"[ Rom. Imp. period, None, None, None, None, un..."
1260,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,{},149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001,"[ 220/19, None, None, -220, -220, exact, BC]"
1944,/text/79501?location=126&patt=&bookid=23&offse...,"IG XII,8",311,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,8\n311",Thasos,{},7,1\n\n\n\n5\n\n,Ἐπικράτης Κτησιφῶντος\nΠυθίων Περιθύμου\nΠίν...,IG-XII-8.csv,79501,"[Thasos, None, None, None, None, string, BC]"
2502,/text/218501?location=1497&patt=&bookid=367&of...,Koptos à Kosseir,163,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Koptos à Kosseir\n163,Eg. — el-Boueib,{},2,1\n,Κάλχων̣\nΠανὶ Χρ(υσόδωτηι).,Koptos---Kosseir.csv,218501,"[ el-Boueib, None, None, None, None, string, BC]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212969,/text/220001?location=1497&patt=&bookid=376&of...,Prose sur pierre,60,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Prose sur pierre\n60,Eg. — Alexandria (El Iskandariya) — 89-91 AD —...,{},23,I.1\nII.2\n\n\n5\n\nIII.7\n\n\n10\n\n\nIV.13\n...,θεοῖς καταχθονίοις.\nΠο[μπ]ηία Μοῦσα ἀνειέρωσε...,Prose-sur-pierre.csv,220001,"[ 89-91 AD , 89, 91, None, 90, range, AD]"
212984,/text/189501?location=473&patt=&bookid=252&off...,IRT,264,Regions\nNorth Africa\nTripolitania,IRT\n264,Tripolitania — Lepcis Magna (Lebda) — 3rd c. A...,{},13,"a,pediment.1\n\nb,pilasters.1\n\n\n\n5\n\nc,ba...",ἀγαθῇ τύχῃ [τῶν κυ]ρίων\nἈσσκληπιάδης {²⁶Ἀσκλη...,IRT.csv,189501,"[ 3rd c. AD , 201, 300, None, 250, cent, AD]"
213826,/text/299001?location=1237&patt=&bookid=736&of...,"IDR III,2",310,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,2\n310",Dacia Sup. — Ulpia Traiana-Sarmizegetusa — 2nd...,{},3,1\n\n,[— Au]rel(ius) S[— — — pro sal(ute) sua]\n[et(...,IDR-III-2.csv,299001,"[ 2nd/3rd c. AD, 101, 200, None, 150, cent, AD]"
214545,/text/279001?location=1090&patt=&bookid=659&of...,"TAM IV,1",181,Regions\nAsia Minor\nBithynia,"TAM IV,1\n181",Bith. — Nikomedeia (Izmit),{},3,1\n\n,— — — Ἀράβιος ὁ θεῖος μου κὲ Μ— — —\n— — — Θεα...,TAM-IV-1.csv,279001,"[ Nikomedeia (Izmit), None, None, None, None, ..."


In [0]:
set_with_dataframe(PHI_overview.add_worksheet("PHI_by_500_dates_v1", 1, 1), PHI_by_500)